In [1]:
import json
import os
import math
import re
import logging
from functools import reduce, partial
from collections import OrderedDict
from typing import Dict, List, Tuple, Union, Callable
import pprint
from mkdir_p import mkdir_p
    
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn.model_selection import ParameterGrid
from flatten_dict import flatten

from nnattack.variables import auto_var
from params import (
    compare_attacks,
    compare_defense,
    #parametric_defense,
    
    #compare_nns,
    
    nn_k1_robustness,
    nn_k3_robustness,
    rf_robustness,
    dt_robustness,
    lr_ap_robustness,
    lr_at_robustness,
    mlp_ap_robustness,
    mlp_at_robustness,
    
    tst_scores,
    
    dt_robustness_figs,
    nn_k1_robustness_figs,
    nn_k3_robustness_figs,
    rf_robustness_figs,
    
    nn1_def,
    nn3_def,
    dt_def,
    rf_def,
    lr_def,
    mlp_def,
)
import params
#import params_l2
from utils import set_plot, get_result, write_to_tex, union_param_key, params_to_dataframe, table_wrapper

auto_var.set_variable_value('random_seed', 0)
auto_var.set_variable_value('ord', 'inf')
auto_var.set_logging_level(0)

compare_attacks = compare_attacks()
compare_defense = compare_defense()
tst_scores = tst_scores()

#compare_nns = compare_nns()
mlp_ap_robustness = mlp_ap_robustness()
mlp_at_robustness = mlp_at_robustness()
lr_ap_robustness = lr_ap_robustness()
lr_at_robustness = lr_at_robustness()
nn_k1_robustness = nn_k1_robustness()
nn_k3_robustness = nn_k3_robustness()
rf_robustness = rf_robustness()
dt_robustness = dt_robustness()
dt_robustness_figs = dt_robustness_figs()
nn_k1_robustness_figs = nn_k1_robustness_figs()
nn_k3_robustness_figs = nn_k3_robustness_figs()
rf_robustness_figs = rf_robustness_figs()

nn1_def = nn1_def()
nn3_def = nn3_def()
dt_def = dt_def()
rf_def = rf_def()
lr_def = lr_def()
mlp_def = mlp_def()


Using TensorFlow backend.
2019-10-02 23:46:15 WARNING  From /home/arbiter/.pyenv/versions/3.7.3/envs/ap-submission/lib/python3.7/site-packages/cleverhans/utils_tf.py:341: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



In [2]:
def result_latex_figs(exp_name, control_var, caption):
    control = ParameterGrid(control_var)
    ret = """
\\begin{figure}[ht!]
\\centering"""
    img_paths = []
    for i, g in enumerate(control):
        dataset, ord = g['dataset'], g['ord']
        img_path = f'./figs/{exp_name}_{dataset}_{ord}.eps'
        dataset = dataset.replace("_", " ")
        ret += """
\\subfloat[%s]{
    \\includegraphics[width=.45\\textwidth]{%s}}""" % (dataset, img_path)
        if i % 2 == 1:
            ret += "\n"
    ret += """
\\caption{%s}
\\label{fig:%s}
\\end{figure} 
""" % (caption, exp_name)
    return ret
                      
def plot_result(df, exp_name, control_var, variables,
                get_title_fn: Union[Callable[[Dict], str], None]=None,
                get_label_name_fn: Union[Callable[[Dict], str], None]=None,
                get_label_color_fn: Union[Callable[[Dict], str], None]=None, show_plot=True):
    ret = []
    for g in ParameterGrid(control_var):
        temp_df = df
                      
        if get_title_fn is None:
            title = exp_name
            for k, v in g.items():
                title = title + f"_{get_var_name(k, v)}"
        else:
            title = get_title_fn(g)
            
        for k, v in g.items():
            temp_df = temp_df.loc[df[k] == v]
                      
        fig, ax = plt.subplots()
        ax.set_title(title, fontsize=20)
        for name, group in temp_df.groupby(variables):
            #print(name, len(group))
            eps_list = [re.findall(r'[-+]?\d*\.\d+|\d+', t)[0] for t in group.mean().index.tolist()[:-1]]
            s = [r for r in group.mean().tolist()[:-1] if not np.isnan(r)]
            x = [float(eps_list[i]) for i, r in enumerate(group.mean().tolist()[:-1]) if not np.isnan(r)]
                      
            if get_label_name_fn is not None:
                label = get_label_name(name)
            elif isinstance(name, str):
                label = get_var_name(variables[0], name)
            else:
                mod_names = []
                for i, n in enumerate(name):
                    mod_names.append(get_var_name(variables[i], n))
                label = mod_name.join("_")

            if get_label_color_fn is not None:
                ax.plot(x, s, label=label, linewidth=3.5, color=get_label_color_fn(name))
            else:
                ax.plot(x, s, label=label, linewidth=3.5)

        dataset = g['dataset']
        ord = g['ord']
        set_plot(fig, ax)
        plt.savefig(f'./figs/{exp_name}_{dataset}_{ord}.eps', format='eps')
        plt.savefig(f'./figs/{exp_name}_{dataset}_{ord}.png', format='png')
        ret.append((g, f'./figs/{exp_name}_{dataset}_{ord}.eps'))
        if show_plot:
            plt.show()
        else:
            plt.close()
    return ret

In [3]:
def parbox(width, content):
    return "\\parbox{%dmm}{\\centering %s}" % (width, content)

def knn_attack_plots(exp_name, grid_param, caption='', show_plot=True):
    df = params_to_dataframe(grid_param)
    datasets = set.union(*[set(g['dataset']) for g in grid_param]) if isinstance(grid_param, list) else grid_param['dataset']

    control = {
        'dataset': datasets,
        'ord': grid_param[0]['ord'],
    }
    variables = ['attack']
    plot_result(df, exp_name, control, variables, show_plot)
    return result_latex_figs(exp_name, control, caption)

def get_var_name(var, arg):
    if var == 'dataset':
        return auto_var.get_var_shown_name(var, arg)
    return arg.replace('_', '-')

def avg_pert_table(exp_name, grid_param, columns, rows, objs:list=None, obj_formats:list=None):
    if objs is None:
        objs = ['avg_pert']
    columns = list(filter(lambda a: a not in ['n_features', 'n_samples', 'n_classes'], columns))
    if len(columns) == 0 or len(rows) == 0:
        return pd.DataFrame({})
    df = params_to_dataframe(grid_param, objs)
    
    d = OrderedDict()
    col_grid = OrderedDict([(c, union_param_key(grid_param, c)) for c in columns])
    row_grid = OrderedDict([(r, union_param_key(grid_param, r)) for r in rows])
    for i, obj in enumerate(objs):
        temp_df = df.groupby(columns + rows)[obj].mean()
        temp_df_sem = df.groupby(columns + rows)[obj].sem()
        
        if obj == 'tst_score':
            assert columns[0] == 'model'
        for col in ParameterGrid(col_grid):
            col_k = tuple(col[c] for c in columns)
            col_name = tuple([get_var_name(c, col[c]) for c in columns[:-1]] \
                             + ["%s-%s" % (get_var_name(columns[-1], col[columns[-1]]), obj.replace("_", "-"))])
            d[col_name] = {}
            for row in ParameterGrid(row_grid):
                row_k = tuple(row[r] for r in rows)
                row_name = tuple(get_var_name(r, row[r]) for r in rows)
                if (col_k + row_k) in temp_df:
                    #d[col_name][row_name] = "$%.3f \pm %.3f$" % (temp_df[col_k + row_k], temp_df_sem[col_k + row_k])
                    if obj_formats is None:
                        str_format = "$%.3f$"
                    else:
                        str_format = obj_formats[i]
                    d[col_name][row_name] = str_format % (temp_df[col_k + row_k])
                    if temp_df[col_k + row_k] < 1:
                        d[col_name][row_name] = d[col_name][row_name].replace("0.", ".")
                else:
                    d[col_name][row_name] = -1

    #d = OrderedDict([(k, d[k]) for k in d.keys()])
    return pd.DataFrame(d)

def dataset_stat_column(df, grid_param, columns, rows):
    if ("n_features" not in columns) and ("n_samples" not in columns) and ("n_classes" not in columns) \
        and ("n_train" not in columns) and ("n_test" not in columns):
        return df
    
    column_names = {
        'n_train': '\# training',
        'n_test': '\# testing',
        'n_features': '\# features',
        'n_samples': '\# examples',
        'n_classes': '\# classes',
    }
    
    d = df.to_dict(into=OrderedDict)
    datasets = union_param_key(grid_param, "dataset")
    if len(d.keys()) > 0:
        first_key = list(d.keys())[0]
        row_len = 1 if isinstance(d[first_key], str) else len(first_key)
        col_len = 1 if isinstance(first_key, str) else len(first_key)
        ori_cols = list(d.keys())
    else:
        row_len = 1
        col_len = 1
        ori_cols = []
    
    for dataset in datasets:
        X, y, _ = auto_var.get_var_with_argument("dataset", dataset)
        row_name = (get_var_name("dataset", dataset), )
        for col in columns:
            if col not in column_names:
                continue
            column_name = tuple(['-' for _ in range(col_len-1)] + [column_names[col]])
            if col == "n_features":
                d.setdefault(column_name, {})[row_name] = X.shape[1]
            elif col == "n_samples":
                d.setdefault(column_name, {})[row_name] = X.shape[0]
            elif col == "n_train":
                d.setdefault(column_name, {})[row_name] = X.shape[0] - 200
            elif col == "n_test":
                d.setdefault(column_name, {})[row_name] = 100
            elif col == "n_classes":
                d.setdefault(column_name, {})[row_name] = len(np.unique(y))
                
    for col in ori_cols:
        d.move_to_end(col)
        
    return pd.DataFrame(d)
    
def cmp_ratio(df):
    ret = OrderedDict()
    d = df.to_dict(into=OrderedDict)
    cmp_base = []
    
    i = 0
    for col, col_dict in d.items():
        ret[col] = col_dict
        if 'avg-pert' not in col[1]:
            continue
        if i == 0 or i == 1:
            cmp_base.append(col_dict)
            i += 1
            continue
        temp = {}
        for k, v in col_dict.items():
            if v == -1 or cmp_base[i % 2][k] == -1:
                temp[k] = int(-1)
            else:
                v = v.replace("$", "")
                t = cmp_base[i % 2][k].replace("$", "")
                temp[k] = "$%.2f$" % (float(v) / float(t))
        
        ret[tuple([c for c in col[:-1]] + ["%s imp." % col[-1]])] = temp
        i += 1
        
    return pd.DataFrame(ret)

def max_imp(df):
    ret = OrderedDict()
    d = df.to_dict(into=OrderedDict)
    
    def add_new_col(col_list, ret):
        new_col = {}
        
        for attack_name in [col_list[0][0][1], col_list[1][0][1]]:
            temp = list(filter(lambda t: t[0][1] == attack_name, col_list))
            imps = []
            for c in temp:
                imps.append([float(v.replace("$", "")) if v != -1 else -1 for _, v in c[1].items()])
            imps = (np.array(imps).T).argmax(axis=1)

            new_col = {}
            new_col_imp = {}
            new_col_eps = {}
            pcol = temp[0][0]
            
            if 'd' in pcol[0].split("-")[-1]:
                tt = pcol[0].split("-")
                tt.pop(-2)
            else:
                tt = pcol[0].split("-")[:-1]
            new_col_name = ("-".join(tt), pcol[1])
            new_col_imp_name = ("-".join(tt), ("%s imp." % pcol[1]))
            new_col_eps_name = ("-".join(tt), ("%s $\\epsilon$" % pcol[1]))
            for i, idx in enumerate(imps):
                k, v = list(temp[idx][1].items())[i]
                new_col[k] = v
                k, v = list(temp[idx][2].items())[i]
                new_col_imp[k] = v 

                if 'd' in temp[idx][0][0].split("-")[-1]:
                    new_col_eps[k] = "$" + ("%.1f$" % (float(temp[idx][0][0].split("-")[-2]) * 0.01))[1:]
                else:
                    new_col_eps[k] = "$" + ("%.1f$" % (float(temp[idx][0][0].split("-")[-1]) * 0.01))[1:]

            ret[new_col_eps_name] = new_col_eps
            ret[new_col_name] = new_col
            ret[new_col_imp_name] = new_col_imp
    
    prev_col = None
    temp = []
    for i, (col, col_dict) in enumerate(d.items()):
        if 'd' in col[0].split("-")[-1]:
            check_idx = -2
        else:
            check_idx = -1
            
        if i == 0 or i == 1:
            ret[col] = col_dict
            continue
            
        if len(temp) == 0:
            temp.append(col_dict)
        elif i % 2 == 1:
            temp[-1] = (prev_col, temp[-1], col_dict)
            if i == (len(d.items())-1):
                add_new_col(temp, ret)
        else:
            if col[0].split("-")[:check_idx] != prev_col[0].split("-")[:check_idx]:
                add_new_col(temp, ret)
                temp = [col_dict]
            else:
                temp.append(col_dict)
                
        prev_col = col
        
    return pd.DataFrame(ret)

def bold_best(df, reverse=False):
    d = df.to_dict(into=OrderedDict)
    
    temp = []
    for i, (col, col_dict) in enumerate(d.items()):
        temp.append([])
        for row, row_value in col_dict.items():
            if isinstance(row_value, str):
                temp[-1].append(float(row_value.replace("$", '')))
            else:
                temp[-1].append(np.inf if reverse else -np.inf)
            
    temp = np.array(temp).T
    if reverse:
        best_idx = temp.argmin(axis=1)
    else:
        best_idx = temp.argmax(axis=1)
        
    ret = OrderedDict()
    for i, (col, col_dict) in enumerate(d.items()):
        ret[col] = {}
        for j, (row, row_value) in enumerate(col_dict.items()):
            if not isinstance(row_value, str):
                ret[col][row] = row_value
            else:
                if float(row_value[1:-1]) == temp[j][best_idx[j]]:
                    ret[col][row] = "$\\mathbf{" + row_value[1:-1] + "}$"
                else:
                    ret[col][row] = row_value

    return pd.DataFrame(ret)

def gen_table(exp_name, grid_params, columns, rows, objs=None,
              combine_method=None, additionals=None, obj_formats=None):
    if objs is None:
        objs = ['avg_pert']
    df = pd.DataFrame({})
    if combine_method is None:
        df = avg_pert_table(exp_name, grid_params, columns, rows, objs, obj_formats)
        if additionals:
            for fn in additionals:
                df = fn(df)
    else:
        dfs = []
        for g in grid_params:
            df = avg_pert_table(exp_name, g, columns, rows, objs, obj_formats)
            if additionals:
                for fn in additionals:
                    df = fn(df)
            dfs.append(df)
        df = pd.concat(dfs, axis=combine_method)
    
    if 'dataset' in rows:
        df = dataset_stat_column(df, grid_param, columns, rows)
    return df


In [4]:
def model_acc(df, grid_param):
    # col = ['model', 'attack']
    ret = OrderedDict()
    tst_df = params_to_dataframe(grid_param, ['tst_score'])

    d = df.to_dict(into=OrderedDict)
    models = set([c[0] for c, _ in d.items()])
    
    prev_col =None
    for i, (col, col_dict) in enumerate(d.items()):
        new_col_dict = OrderedDict({})
        if i == 0:
            for row, _ in col_dict.items():
                temp_df = tst_df[(tst_df['model'] == col[0].replace("-", "_"))
                                 & (tst_df['attack'] == 'blackbox') 
                                 & (tst_df['dataset'] == row[0].replace("-", "_"))]
                new_col_dict[row] = "$%.2f$" % temp_df['tst_score'].mean()
            ret[(col[0], col[1].replace('-avg-pert', ' tst acc.'))] = new_col_dict
            
        elif '\\epsilon' in col[1]:
            m = re.match(r"(?P<attack>[a-zA-Z_0-9'-]+) \$\\epsilon\$", col[1])
            attack_name = m.group("attack")[:-9].replace("-", "_") # remove '$epsilon$'
            for row, row_val in col_dict.items():
                if 'd' in col[0].split('-')[-1]:
                    model_name = '%s-%d-%s' % ('-'.join(col[0].split('-')[:-1]),
                                               int(float(row_val.replace("$", ""))*100),
                                               col[0].split('-')[-1],)
                else:
                    model_name = "%s-%d" % (col[0], int(float(row_val.replace("$", ""))*100))
                model_name = model_name.replace("-", "_")
                temp_df = tst_df[(tst_df['model'] == model_name)
                                 & (tst_df['attack'] == attack_name) 
                                 & (tst_df['dataset'] == row[0].replace("-", "_"))]
                new_col_dict[row] = "$%.2f$" % temp_df['tst_score'].mean()
            ret[(col[0], col[1].replace('-avg-pert $\\epsilon$', ' tst acc.'))] = new_col_dict
            
        prev_col = col
        ret[col] = col_dict
    return pd.DataFrame(ret)

In [7]:
def improvement(df):
    ret = OrderedDict()
    d = df.to_dict(into=OrderedDict)
    
    for i, (col, col_dict) in enumerate(d.items()):
        if i != 0:
            ret[col] = {}
        for row, row_value in col_dict.items():
            if i == 0:
                ref = col_dict
                value = 1.0
            elif ref[row] == -1 or row_value == -1:
                value = -1.
            else:
                value = (float(row_value.replace("$", '')) / float(ref[row].replace("$", '')))
                
            if i != 0:
                ret[col][row] = "$%.2f$" % value
        
    return pd.DataFrame(ret)

_, exp_name, grid_param, _ = compare_defense()
avg_caption = """
The \defenderscore across four nonparametric classifiers and corresponding competitors.
A number greater than one indicates that the defense yields a more robust model, 
while less than one indicates less robustness (higher is better; best is in bold).
The \defenderscore for undefended classifiers are always one.
"""
#del grid_param[1]

table_str = gen_table(
                exp_name, grid_param, ['model', 'attack'], ['dataset'], combine_method=1,
                objs=['avg_pert'], additionals=[improvement, bold_best]
            ).to_latex(escape=False)
table_str = re.sub(r"[\s]*adv-nn-k1-30 & [\s]*robustv2-nn-k1-30 & [\s]*advPruning-nn-k1-30",
                   r"\\multicolumn{3}{c}{1-NN}", table_str)
table_str = re.sub(r"[\s]*adv-nn-k3-30 & [\s]*advPruning-nn-k3-30",
                   r"\\multicolumn{2}{c}{3-NN}", table_str)
table_str = re.sub(r"[\s]*adv-decision-tree-d5-30 & [\s]*robust-decision-tree-d5-30 & [\s]*advPruning-decision-tree-d5-30",
                   r"\\multicolumn{3}{c}{DT}", table_str)
table_str = re.sub(r"[\s]*adv-rf-100-30-d5 & [\s]*robust-rf-100-30-d5 & [\s]*advPruning-rf-100-30-d5",
                   r"\\multicolumn{3}{c}{RF}", table_str)
#table_str = re.sub(r"[\s]*adv-mlp-30 & [\s]*advPruning-mlp-30",
#                   r" \\multicolumn{2}{c}{MLP}", table_str)
#table_str = re.sub(r"[\s]*adv-logistic-regression-30 & [\s]*advPruning-logistic-regression-30",
#                   r" \\multicolumn{2}{c}{LR}", table_str)
table_str = table_str.replace("RBA-Exact-KNN-k1-avg-pert & RBA-Exact-KNN-k1-avg-pert & RBA-Exact-KNN-k1-avg-pert",
                              "AT & Wang's & AP")
table_str = table_str.replace("RBA-Approx-KNN-k1-50-avg-pert & RBA-Approx-KNN-k1-50-avg-pert & RBA-Approx-KNN-k1-50-avg-pert",
                              "AT & Wang's & AP")
table_str = table_str.replace("RBA-Approx-KNN-k3-50-avg-pert & RBA-Approx-KNN-k3-50-avg-pert",
                              "AT & AP")
table_str = re.sub(r"[\s]*RBA-Exact-DT-avg-pert & [\s]*RBA-Exact-DT-avg-pert & [\s]*RBA-Exact-DT-avg-pert",
                    " AT & RS & AP", table_str)
table_str = table_str.replace("RBA-Approx-RF-100-avg-pert & RBA-Approx-RF-100-avg-pert & RBA-Approx-RF-100-avg-pert",
                              "AT & RS & AP")
table_str = re.sub(r"[\s]*pgd-avg-pert & [\s]*pgd-avg-pert",
                    " AT & AP", table_str)
table_str = table_str.replace("adv-nnopt-k1-all-avg-pert", "AT")
table_str = table_str.replace("advPruning-decision-tree-d5-30", "AP")
#table_str = table_str.replace("llllllllllllllll", "l|ccc|cc|ccc|ccc||cc|cc")
table_str = table_str.replace("llllllllllll", "l|ccc|cc|ccc|ccc")
write_to_tex(table_str, exp_name + '_table.tex')

./results/fashion-mnist35f-pca25-adv-nn-k1-30-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist
./results/fashion-mnist06f-pca25-adv-nn-k1-30-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist
./results/mnist17f-pca25-adv-nn-k1-30-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist
./results/covtypebin-2200-adv-nn-k1-30-RBA-Approx-KNN-k1-50-rs0-linf.json doesn't exist
./results/fashion-mnist06f-pca25-adv-nn-k3-30-RBA-Approx-KNN-k3-50-rs0-linf.json doesn't exist


In [6]:
def attack_table(exp_name, grid_param):
    table_str = gen_table(
                    exp_name, grid_param, ['model', 'attack'], ['dataset'], combine_method=1,
                    objs=['avg_pert'], additionals=[partial(bold_best, reverse=True)]
                ).to_latex(escape=False)
    table_str = re.sub(r"([a-zA-Z_0-9'-]+) imp\.", "imp.", table_str)
    table_str = re.sub(r"([a-zA-Z_0-9'-]+) tst acc\.", "tst acc.", table_str)
    table_str = re.sub(r"([a-zA-Z_0-9'-]+)-avg-pert", r"\1", table_str)
    table_str = re.sub(r"([a-zA-Z_0-9'-]+) \$\\epsilon\$", r"$\\epsilon$", table_str)
    table_str = table_str.replace("lllllllllllllll", "l|ccccc|cccc|ccc|cc")
    table_str = table_str.replace("-avg-pert", "")
    table_str = table_str.replace("direct-k1", "Direct")
    table_str = table_str.replace("direct-k3", "Direct")
    table_str = table_str.replace("kernelsub-c1000-pgd", "Kernel")
    table_str = table_str.replace("RBA-Exact-KNN-k1", "RBA-Exact")
    table_str = table_str.replace("RBA-Exact-DT", "RBA-Exact")
    table_str = table_str.replace("RBA-Approx-KNN-k1-50", "RBA-Approx")
    table_str = table_str.replace("RBA-Approx-KNN-k3-50", "RBA-Approx")
    table_str = table_str.replace("RBA-Approx-RF-100", "RBA-Approx")
    table_str = table_str.replace("dt-papernots", "Papernot's")
    table_str = table_str.replace("rev-nnopt-k3-50-region", "RBA-Approx")
    table_str = table_str.replace("rf-attack-rev-100", "RBA-Approx")
    table_str = table_str.replace("dt-attack-opt", "RBA-Exact")
    table_str = table_str.replace("decision-tree-d5", "DT")
    table_str = table_str.replace("random-forest-100-d5", "RF")
    table_str = table_str.replace("knn1", "1-NN")
    table_str = table_str.replace("knn3", "3-NN")
    table_str = table_str.replace("\multicolumn{5}{l}", "\multicolumn{5}{c}")
    table_str = table_str.replace("\multicolumn{4}{l}", "\multicolumn{4}{c}")
    table_str = table_str.replace("\multicolumn{3}{l}", "\multicolumn{3}{c}")
    table_str = table_str.replace("\multicolumn{2}{l}", "\multicolumn{2}{c}")
    table_str = table_str.replace("blackbox", "BBox")
    table_str = table_str.replace("RBA-Exact", "\\makecell{RBA\\\\Exact}")
    table_str = table_str.replace("RBA-Approx", "\\makecell{RBA\\\\Approx}")
    return table_str

_, exp_name, grid_param, _ = compare_attacks()
write_to_tex(attack_table(exp_name, grid_param), exp_name + '_table.tex')
#_, exp_name, grid_param, _ = params_l2.compare_attacks()()
#write_to_tex(attack_table(exp_name, grid_param), exp_name + '_table.tex')

In [9]:
exp_fns = [nn_k1_robustness_figs, nn_k3_robustness_figs, dt_robustness_figs, rf_robustness_figs,
           params.nn_k1_approx_robustness_figs()]
model_names = ["1-NN", "3-NN", "Decision tree", "Random forest", "1-NN"]
def get_label_name(name):
    if 'advPruning' in name:
        return "AP"
    elif 'robust' in name:
        return "RS"
    elif 'decision_tree' in name:
        return "Reg."
    elif 'knn1' in name:
        return "Reg."
    elif 'knn3' in name:
        return "Reg."
    elif 'random_forest' in name:
        return "Reg."
        
    return name

def get_label_color(name):
    if 'advPruning' in name:
        return "#ff7f0e"
    elif 'robust' in name:
        return "#1f77b4"
    elif 'decision_tree' in name:
        return "#7f7f7f"
    elif 'knn1' in name:
        return "#7f7f7f"
    elif 'knn3' in name:
        return "#7f7f7f"
    elif 'random_forest' in name:
        return "#7f7f7f"
        
    return name

def compare_nn_plots(exp_name, grid_param, caption='', show_plot=False):
    df = params_to_dataframe(grid_param)
    datasets = set.union(*[set(g['dataset']) for g in grid_param]) if isinstance(grid_param, list) else grid_param['dataset']

    control = {
        'dataset': datasets,
        'ord': grid_param['ord'],
    }
    variables = ['model']
    
    fig_paths = plot_result(df, exp_name, control, variables,
                            get_title_fn=lambda g: get_var_name("dataset", g['dataset']),
                            get_label_name_fn=get_label_name,
                            get_label_color_fn=get_label_color,
                            show_plot=show_plot)
    return fig_paths

def fig_paths_latex(fig_paths: List[List[Tuple[Dict, str]]], fig_label, caption):
    ret = """
\\begin{figure}[ht!]
\\centering"""
    img_paths = []
    for row in fig_paths:
        for entry in row:
            g, img_path = entry
            ret += """
\\subfloat[%s]{
    \\includegraphics[width=%.2f\\textwidth]{%s}}""" % (g['subfig_label'], 1/len(fig_paths[0]), img_path)
        ret += "\n"
    ret += """
\\caption{%s}
\\label{fig:%s}
\\end{figure} 
""" % (caption, fig_label)
    return ret

fig_paths = []
for i, fn in enumerate(exp_fns):
    _, exp_name, grid_param, _ = fn()
    fig_path = compare_nn_plots(exp_name, grid_param, show_plot=False)
    for g, _ in fig_path:
        g['subfig_label'] = model_names[i]
        g['subfig_label'] = get_var_name("dataset", g['subfig_label'])
    fig_paths.append(fig_path)
transpose = [list() for c in fig_paths[0]]
for i, col in enumerate(fig_paths):
    for j, r in enumerate(col):
        transpose[j].append(r)
        
caption = "The maximum perturbation distance allowed versus accuracy."
fig_str = fig_paths_latex(transpose[:5], "defense-cmp", caption)
write_to_tex(fig_str, 'defense_cmp_fig.tex')

#fig_str = fig_paths_latex(transpose[5:], "defense-cmp2", caption)
#write_to_tex(fig_str, 'defense_cmp2_fig.tex')

./results/covtypebin-2200-advPruning-nn-k1-30-RBA-Approx-KNN-k1-50-rs0-linf.json doesn't exist


In [8]:
#_, exp_name, grid_param, _ = compare_nns()
#
#def get_title_fn(g):
#    ret = get_var_name("dataset", g['dataset'])
#    return ret
#
#def compare_nn_plots(exp_name, grid_param, caption='', show_plot=True):
#    df = params_to_dataframe(grid_param)
#    datasets = set.union(*[set(g['dataset']) for g in grid_param]) if isinstance(grid_param, list) else grid_param['dataset']
#
#    control = {
#        'dataset': datasets,
#        'ord': grid_param[0]['ord'],
#    }
#    variables = ['model']
#    figs = plot_result(df, exp_name, control, variables, get_title_fn=get_title_fn, show_plot=show_plot)
#    fig_paths = []
#    for i, f in enumerate(figs):
#        if i % 3 == 0:
#            fig_paths.append([])
#        f[0]['subfig_label'] = f[0]['dataset']
#        f[0]['subfig_label'] = get_var_name("dataset", f[0]['subfig_label'])
#        fig_paths[-1].append(f)
#    
#    return fig_paths_latex(fig_paths, exp_name, caption=caption)
#caption = """
#The maximum perturbation distance allowed versus accuracy with different $k$ of $k$-NN classifier
#using RBA-Approx attack searching 50 regions."
#"""
#fig_str = compare_nn_plots(exp_name, grid_param, caption=caption, show_plot=False)
#write_to_tex(fig_str, exp_name + '_fig.tex')

In [9]:
from params import datasets, tree_datasets
_, _, grid_param, _ = tst_scores()

col_names = [
    "\\parbox{15mm}{\\centering \# training \\\\ (1-NN, 3-NN)}",
    "\\parbox{15mm}{\\centering \# training \\\\ (DT, RF, MLP)}",
    "\\parbox{15mm}{\\centering \# testing \\\\ (perturbation)}",
    "\\parbox{15mm}{\\centering \# testing \\\\ (test accuracy)}",
    "\# features",
    "\# classes",
]
ret = OrderedDict()
for i, ds in enumerate(datasets):
    X, y, _ = auto_var.get_var_with_argument("dataset", ds)
    tX, _, _ = auto_var.get_var_with_argument("dataset", tree_datasets[i])
    ret[auto_var.get_var_shown_name("dataset", ds)] = OrderedDict([
        (col_names[0], X.shape[0]-200),
        (col_names[1], tX.shape[0]-200),
        (col_names[2], 100),
        (col_names[3], 200),
        (col_names[4], X.shape[1]),
        (col_names[5], 2),
    ])
df = pd.DataFrame(ret).T
df = df[[c for c in col_names]]

exp_name = "dataset-stats"
caption = "Dataset statistics."
table_str = table_wrapper(df, table_name=exp_name, caption=caption)
table_str = table_str.replace("lrrrrrr", "lcccccc")
write_to_tex(table_str, exp_name + '_table.tex')

ValueError: too many values to unpack (expected 3)

In [11]:
def structure_fonts(series):
    if '\\# train' in series.name[1]:
        return series.apply(lambda x: "$%d$" % x if not np.isnan(x) else "-1")
    else:
        return series.apply(lambda x: ("$%.3f$" % x) if x >= 1 else ("$%.3f$" % x).replace("0.", "."))
    
def preprocess(grid_param):
    models = [i.replace("_", "-") for i in union_param_key(grid_param, 'model')]
    attacks = []
    for i in union_param_key(grid_param, 'attack'):
        for s in ['-avg-pert', '-tst-score', '-aug-len', '-imp']:
            attacks.append(i.replace("_", "-") + s)
    col_names = []
    for model in models:
        for attack in attacks:
            if '-imp' in attack and model == models[0]:
                continue
            col_names.append((model, attack))
    return col_names

def process(task_fn):
    _, exp_name, grid_param, _ = task_fn()
    df = gen_table(exp_name, grid_param, ['model', 'attack'], ['dataset'],
                   combine_method=1, objs=['tst_score', 'avg_pert', 'aug_len'],
                   additionals=[])

    models = [i.replace("_", "-") for i in union_param_key(grid_param, 'model')]
    attack = grid_param[0]['attack'][0].replace("_", '-')
    col_names = preprocess(grid_param)

    df = df.apply(lambda a: a.apply(lambda b: float(str(b).replace("$", "")) if b else b))
    for model in models[1:]:
        df[(model, attack + '-imp')] = df[(model, attack + '-avg-pert')] / df[(models[0], attack + '-avg-pert')]
    df = df[col_names]
    df = df.rename(index=str, columns={
        attack + "-aug-len": "\# train",
        attack + "-tst-score": parbox(8, "test \\\\ accuracy"),
        attack + "-avg-pert": parbox(9, "ER"),
        attack + "-imp": "\\defenderscore",
    })
    return df

def postprocess(task_fn, df, rename_columns=None, caption=None):
    _, exp_name, grid_param, _ = task_fn()
    if rename_columns:
        df = df.rename(index=str, columns=rename_columns)
    df = df.apply(structure_fonts)
    table_str = table_wrapper(df, table_name=exp_name, caption=caption)
    #table_str = df.to_latex(escape=False)
    table_str = table_str.replace("{l}", "{c}")
    table_str = table_str.replace("llllllllllllllll", "lccc|cccc|cccc|cccc")
    table_str = table_str.replace("$.000$", "-")
    table_str = table_str.replace("$nan$", "-")
    table_str = table_str.replace("begin{table}", "begin{table*}")
    table_str = table_str.replace("end{table}", "end{table*}")
    return table_str

In [12]:
caption_template = """
The number of training data left after adversarial pruning (AP), testing accuracy, empirical robustness,
and \\defenderscore with different separation parameter of AP for {}.
"""
"""
Testing accuracy is a sanity check that we are not giving away all accuracy for robustness.
The higher the empirical robustness is means the classifier is more robust to the given attack.
When considering the strength of the attack, empirical robustness is lower the better.
When considering the strength of the defense, \\defenderscore is higher the better.
For \\defenderscore higher mean that after defense (AP), the classifier become more robust, thus higher the better.
"""

fn = nn_k1_robustness
df = process(fn)
caption = caption_template.format("1-NN")
rename_columns = {
    "knn1": "1-NN",
    "advPruning-nn-k1-10": "AP (separation parameter $r$=.1)",
    "advPruning-nn-k1-30": "AP (separation parameter $r$=.3)",
    "advPruning-nn-k1-50": "AP (separation parameter $r$=.5)",
}
_, exp_name, _, _ = fn()
table_str = postprocess(fn, df, rename_columns, caption)
write_to_tex(table_str, exp_name + '_table.tex')

fn = nn_k3_robustness
df = process(fn)
caption = caption_template.format("3-NN")
rename_columns = {
    "knn3": "3-NN",
    "advPruning-nn-k3-10": "AP (separation parameter $r$=.1)",
    "advPruning-nn-k3-30": "AP (separation parameter $r$=.3)",
    "advPruning-nn-k3-50": "AP (separation parameter $r$=.5)",
}
_, exp_name, _, _ = fn()
table_str = postprocess(fn, df, rename_columns, caption)
write_to_tex(table_str, exp_name + '_table.tex')

fn = dt_robustness
df = process(fn)
caption = caption_template.format("DT")
rename_columns = {
    "decision-tree-d5": "DT",
    "advPruning-decision-tree-d5-10": "AP (separation parameter $r$=.1)",
    "advPruning-decision-tree-d5-30": "AP (separation parameter $r$=.3)",
    "advPruning-decision-tree-d5-50": "AP (separation parameter $r$=.5)",
}
_, exp_name, _, _ = fn()
table_str = postprocess(fn, df, rename_columns, caption)
write_to_tex(table_str, exp_name + '_table.tex')

fn = rf_robustness
df = process(fn)
caption = caption_template.format("RF")
rename_columns = {
    "random-forest-100-d5": "RF",
    "advPruning-rf-100-10-d5": "AP (separation parameter $r$=.1)",
    "advPruning-rf-100-30-d5": "AP (separation parameter $r$=.3)",
    "advPruning-rf-100-50-d5": "AP (separation parameter $r$=.5)",
}
_, exp_name, _, _ = fn()
table_str = postprocess(fn, df, rename_columns, caption)
write_to_tex(table_str, exp_name + '_table.tex')

#fn = lr_ap_robustness
#df = process(fn)
#caption = caption_template.format("LR")
#rename_columns = {
#    "logistic-regression": "LR",
#    "advPruning-logistic-regression-10": "AP (separation parameter $r$=.1)",
#    "advPruning-logistic-regression-30": "AP (separation parameter $r$=.3)",
#    "advPruning-logistic-regression-50": "AP (separation parameter $r$=.5)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')
#
#fn = mlp_ap_robustness
#df = process(fn)
#caption = caption_template.format("MLP")
#rename_columns = {
#    "mlp": "MLP",
#    "advPruning-mlp-10": "AP (separation parameter $r$=.1)",
#    "advPruning-mlp-30": "AP (separation parameter $r$=.3)",
#    "advPruning-mlp-50": "AP (separation parameter $r$=.5)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')

./results/fashion-mnist35f-pca25-advPruning-nn-k1-10-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist
./results/fashion-mnist35f-pca25-advPruning-nn-k1-50-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist
./results/mnist17f-pca25-advPruning-nn-k1-10-RBA-Exact-KNN-k1-rs0-linf.json doesn't exist


In [11]:
def process2(task_fn):
    _, exp_name, grid_param, _ = task_fn()
    df = gen_table(exp_name, grid_param, ['model', 'attack'], ['dataset'],
                   combine_method=1, objs=['tst_score', 'avg_pert'], additionals=[])

    models = [i.replace("_", "-") for i in union_param_key(grid_param, 'model')]
    attack = grid_param[0]['attack'][0].replace("_", '-')
    col_names = [a for a in preprocess(grid_param) if '-aug-len' not in a[1]]

    df = df.apply(lambda a: a.apply(lambda b: float(str(b).replace("$", "")) if b else b))
    for model in models[1:]:
        df[(model, attack + '-imp')] = df[(model, attack + '-avg-pert')] / df[(models[0], attack + '-avg-pert')]
    df = df[col_names]
    df = df.rename(index=str, columns={
        attack + "-tst-score": parbox(8, "testing \\\\ accuracy"),
        attack + "-avg-pert": parbox(9, "empirical \\\\ robustness"),
        attack + "-imp": "\\defenderscore",
    })
    return df

def postprocess2(task_fn, df, rename_columns, caption):
    _, exp_name, grid_param, _ = task_fn()
    df = df.rename(index=str, columns=rename_columns)
    df = df.apply(structure_fonts)
    table_str = table_wrapper(df, table_name=exp_name, caption=caption,)
    #table_str = df.to_latex(escape=False)
    table_str = table_str.replace("{l}", "{c}")
    table_str = table_str.replace("llllllllllll", "lcc|ccc|ccc|ccc")
    table_str = table_str.replace("$.000$", "-")
    table_str = table_str.replace("$nan$", "-")
    return table_str

In [12]:
caption_template = """
The testing accuracy, empirical robustness,
and \\defenderscore with differen attack distance of adversarial training (AT) for {}.
"""
"""
Testing accuracy is a sanity check that we are not giving away all accuracy for robustness.
The higher the empirical robustness is means the classifier is more robust to the given attack.
For \\defenderscore higher mean that after defense (AP), the classifier become more robust, thus higher the better.
"""

#fn = mlp_at_robustness
#df = process2(fn)
#caption = caption_template.format("MLP")
#rename_columns = {
#    "mlp": "MLP",
#    "adv-mlp-10": "AT (attack distance $r$=.1)",
#    "adv-mlp-30": "AT (attack distance $r$=.3)",
#    "adv-mlp-50": "AT (attack distance $r$=.5)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess2(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')
#
#fn = lr_at_robustness
#df = process2(fn)
#caption = caption_template.format("LR")
#rename_columns = {
#    "logistic-regression": "LR",
#    "adv-logistic-regression-10": "AT (attack distance $r$=.1)",
#    "adv-logistic-regression-30": "AT (attack distance $r$=.3)",
#    "adv-logistic-regression-50": "AT (attack distance $r$=.5)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess2(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')
#
#
#fn = params_l2.mlp_at_robustness()
#df = process2(fn)
#caption = caption_template.format("MLP")
#rename_columns = {
#    "mlp": "MLP",
#    "adv-mlp-25": "AT (attack distance $r$=.25)",
#    "adv-mlp-50": "AT (attack distance $r$=.50)",
#    "adv-mlp-75": "AT (attack distance $r$=.75)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess2(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')
#
#fn = params_l2.lr_at_robustness()
#df = process2(fn)
#caption = caption_template.format("LR")
#rename_columns = {
#    "logistic-regression": "LR",
#    "adv-logistic-regression-25": "AT (attack distance $r$=.25)",
#    "adv-logistic-regression-50": "AT (attack distance $r$=.50)",
#    "adv-logistic-regression-75": "AT (attack distance $r$=.75)",
#}
#_, exp_name, _, _ = fn()
#table_str = postprocess2(fn, df, rename_columns, caption)
#write_to_tex(table_str, exp_name + '_table.tex')

In [17]:
def def_process(task_fn):
    _, exp_name, grid_param, _ = task_fn()
    df = gen_table(exp_name, grid_param, ['model', 'attack'], ['dataset'],
                   combine_method=0, objs=['tst_score', 'avg_pert', 'aug_len'],
                   additionals=[])

    models = [i.replace("_", "-") for i in union_param_key(grid_param, 'model')]
    attack = grid_param[0]['attack'][0].replace("_", '-')
    col_names = preprocess(grid_param)

    df = df.apply(lambda a: a.apply(lambda b: float(str(b).replace("$", "")) if b else b))
    for model in models[1:]:
        df[(model, attack + '-imp')] = df[(model, attack + '-avg-pert')] / df[(models[0], attack + '-avg-pert')]
    df = df[col_names]
    df = df.rename(index=str, columns={
        attack + "-aug-len": "\# train",
        attack + "-tst-score": parbox(8, "test \\\\ accuracy"),
        attack + "-avg-pert": parbox(9, "ER"),
        attack + "-imp": "\\defenderscore",
    })
    return df


#ds = ['australian', 'cancer', 'covtype', 'diabetes', 'f-mnist06', 'f-mnist35', 'fourclass', 'halfmoon', 'mnist17']
#table = OrderedDict()
#ds_eps = params_l2.ds_eps
#
#df = def_process(params_l2.nn1_def())
#ds_def = {'AT': np.zeros(len(ds)), "Wang's": np.zeros(len(ds)), 'AP': np.zeros(len(ds))}
#for idx in df['knn1'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-nn-k1-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def["Wang's"][ds_idx] = df[f'robustv2-nn-k1-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-nn-k1-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#table['1-NN'] = ds_def
#
#df = def_process(params_l2.nn3_def())
#ds_def = {'AT': np.zeros(len(ds)), 'AP': np.zeros(len(ds))}
#for idx in df['knn3'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-nn-k3-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-nn-k3-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#table['3-NN'] = ds_def
#
#df = def_process(params_l2.rf_def())
#ds_def = OrderedDict([('AT', np.zeros(len(ds))), ('RS', np.zeros(len(ds))), ('AP', np.zeros(len(ds)))])
#for idx in df['random-forest-100-d5'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-rf-100-{ds_eps[ds_name]}-d5']['\defenderscore'][ds_name][0]
#    ds_def['RS'][ds_idx] = df[f'robust-rf-100-{ds_eps[ds_name]}-d5']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-rf-100-{ds_eps[ds_name]}-d5']['\defenderscore'][ds_name][0]
#table['RF'] = ds_def
#
#df = def_process(params_l2.dt_def())
#ds_def = OrderedDict([('AT', np.zeros(len(ds))), ('RS', np.zeros(len(ds))), ('AP', np.zeros(len(ds)))])
#for idx in df['decision-tree-d5'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-decision-tree-d5-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['RS'][ds_idx] = df[f'robust-decision-tree-d5-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-decision-tree-d5-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#table['DT'] = ds_def
#
#df = def_process(params_l2.lr_def())
#ds_def = OrderedDict([('AT', np.zeros(len(ds))), ('AP', np.zeros(len(ds)))])
#for idx in df['logistic-regression'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-logistic-regression-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-logistic-regression-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#table['LR'] = ds_def
#
#df = def_process(params_l2.mlp_def())
#ds_def = OrderedDict([('AT', np.zeros(len(ds))), ('AP', np.zeros(len(ds)))])
#for idx in df['mlp'].index:
#    ds_name = idx[0]
#    ds_idx = ds.index(idx[0])
#    ds_def['AT'][ds_idx] = df[f'adv-mlp-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#    ds_def['AP'][ds_idx] = df[f'advPruning-mlp-{ds_eps[ds_name]}']['\defenderscore'][ds_name][0]
#table['MLP'] = ds_def
#
#df = pd.DataFrame(flatten(table), index=ds)
#df

./results/mnist17-2200-pca25-adv-nn-k1-75-RBA-Exact-KNN-k1-rs0-l2.json doesn't exist
./results/australian-adv-nn-k3-50-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/diabetes-adv-nn-k3-25-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/diabetes-robustv2-nn-k3-25-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/fashion-mnist35-2200-pca25-adv-nn-k3-75-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/fashion-mnist06-2200-pca25-adv-nn-k3-75-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/fashion-mnist06-2200-pca25-robustv2-nn-k3-75-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/mnist17-2200-pca25-robustv2-nn-k3-75-RBA-Approx-KNN-k3-50-rs0-l2.json doesn't exist
./results/fourclass-robust-rf-100-25-d5-RBA-Approx-RF-100-rs0-l2.json doesn't exist
./results/diabetes-robust-rf-100-25-d5-RBA-Approx-RF-100-rs0-l2.json doesn't exist
./results/covtypebin-10200-robust-rf-100-50-d5-RBA-Approx-RF-100-rs0-l2.json doesn't exist
./results/diabetes-adv-

1-NN                          3-NN                  RF  \
                  AT    Wang's        AP        AT        AP        AT   
australian  1.190647  1.607914  1.607914 -2.252252  1.200450  1.041441   
cancer      1.067692  1.040000  1.403077  0.979339  1.351240  1.008837   
covtype     1.280899  2.808989  2.808989  1.056391  2.218045  1.173010   
diabetes    1.000000  1.855263  1.855263 -6.451613  1.496774  1.077181   
f-mnist06   1.067164  2.305970  2.305970 -4.149378  1.597510  0.804813   
f-mnist35   0.776163  1.078488  1.078488 -2.469136  1.034568  0.634660   
fourclass   0.971963  3.261682  3.261682  0.823009  2.991150  0.866667   
halfmoon    1.075758  3.106061  3.106061  0.916667  2.120370  0.950920   
mnist17    -3.225806  1.158065  1.158065  0.729947  1.082888  0.644295   

                                       DT                             LR  \
                  RS        AP         AT         RS        AP        AT   
australian  0.963964  1.050450   2.120000   5.840000  3.373333  3.063745   
cancer      1.023564  1.235641   0.912052   1.130293  1.100977  1.448517   
covtype    -3.460208  2.117647   1.000000   5.028169  4.802817  4.207547   
diabetes   -3.355705  1.197987 -10.204082 -10.204082  1.336735  1.820388   
f-mnist06   0.831551  1.566845   0.840909   0.000000  1.795455  1.184783   
f-mnist35   1.550351  1.063232   1.328000   2.624000  1.856000  1.266055   
fourclass  -6.666667  3.420000   1.437500   1.194444  2.909722  1.348921   
halfmoon    1.226994  1.834356   1.033708   1.022472  1.719101  0.823256   
mnist17     0.288591  1.020134   1.204545   2.469697  1.356061  1.166667   

                           MLP            
                  AP        AT        AP  
australian  1.382470  9.397436  5.371795  
cancer      1.024433  1.847495  1.418301  
covtype     2.141509  3.728571  3.871429  
diabetes    2.873786  3.846774  1.548387  
f-mnist06   1.713768  2.077922  3.168831  
f-mnist35   1.103976  0.862385  1.018349  
fourclass   1.366906  2.142857  2.590909  
halfmoon    1.088372  1.447761  1.701493  
mnist17     1.056667  1.056022  0.913165

In [18]:
def to_str(s):
    max_value = s.max()
    ret = s.apply(lambda x: '$%.2f$' % x if x != max_value else '$\\mathbf{%.2f}$' % x)
    return ret
df = df.fillna(-1)
for model_name in ['1-NN', '3-NN', 'DT', 'LR', 'MLP', 'RF']:
    df[model_name] = df[model_name].apply(to_str, axis=1)
table_str = df.to_latex(escape=False)
table_str = table_str.replace("{l}", "{c}")
table_str = table_str.replace("llllllllllllllll", "lccc|cc|ccc|ccc|cc|cc")
table_str = table_str.replace("$.000$", "-")
table_str = table_str.replace("$nan$", "-")

write_to_tex(table_str, 'compare_defense_table_l2.tex')

In [23]:
_, exp_name, grid_params, _ = params.fullds()()
df = params_to_dataframe(grid_params, columns=[("tst_score"), ("avg_pert"), "aug_len"])
table = {}
mnist_model_names = ['random_forest_500_d10', 'approxAP_rf_500_20_d10',
                     'knn1', 'approxAP_nn_k1_20',
                     'knn3', 'approxAP_nn_k3_20',
                     'xgb_xgb_models/fullmnist_pca100.unrob.0200.model',
                     'xgb_xgb_models/prune_fullmnist_pca100_20_linf.unrob.0200.model',
                    ]
fashion_model_names = ['random_forest_500_d10', 'approxAP_rf_500_20_d10',
                       'knn1', 'approxAP_nn_k1_20',
                       'knn3', 'approxAP_nn_k3_20',
                       'xgb_xgb_models/fullfashion_pca100.unrob.0200.model',
                       'xgb_xgb_models/prune_fullfashion_pca100_20_linf.unrob.0200.model',
                      ]
shown_names = ['RF', 'AP RF', '$1$-NN', 'AP $1$-NN', '$3$-NN', 'AP $3$-NN']
for ds_name_t, d in df.groupby("dataset"):
    ds_name = ds_name_t.split("_")[0]
    if 'mnist' in ds_name:
        model_names = mnist_model_names
    else:
        model_names = fashion_model_names
 
    table[ds_name] = []
    for i, model_name in enumerate(model_names):
        model_df = d[d['model'] == model_name]
        if len(model_df['tst_score'].values) == 0:
            table[ds_name].append("%.3f" % -1)
            table[ds_name].append("%.3f" % -1)
            if i % 2:
                table[ds_name].append("%.3f" % -1)
        else:
            table[ds_name].append("%.3f" % model_df['tst_score'].values[0])
            table[ds_name].append("%.3f" % model_df['avg_pert'].values[0])
            if i % 2:
                try:
                    table[ds_name].append(
                        "%.3f" % (model_df['avg_pert'].values[0] / d[d['model'] == model_names[i-1]]['avg_pert'].values[0]))
                except:
                    table[ds_name].append("%.3f" % -1)
    #for i, model_name in enumerate(model_names):
    #    model_df = d[d['model'] == model_name]
    #    table[ds_name].append("%.3f" % model_df['tst_score'].values[0])
    #    table[ds_name].append("%.3f" % model_df['avg_pert'].values[0])
    #    if i % 2:
    #        table[ds_name].append(
    #            "%.3f" % (model_df['avg_pert'].values[0] / d[d['model'] == model_names[i-1]]['avg_pert'].values[0]))
table = pd.DataFrame.from_dict(table, orient='index', columns=[
    ('RF', 'natural', 'acc.'), ('RF', 'natural', 'ER'),
    ('RF', 'AP', 'acc.'), ('RF', 'AP', 'ER'), ('RF', 'AP', '\\defscore'),
    ('$1$-NN', 'natural', 'acc.'), ('$1$-NN', 'natural', 'ER'),
    ('$1$-NN', 'AP', 'acc.'), ('$1$-NN', 'AP', 'ER'), ('$1$-NN', 'AP', '\\defscore'),
    ('$3$-NN', 'natural', 'acc.'), ('$3$-NN', 'natural', 'ER'),
    ('$3$-NN', 'AP', 'acc.'), ('$3$-NN', 'AP', 'ER'), ('$3$-NN', 'AP', '\\defscore'),
    ('GBM', 'natural', 'acc.'), ('GBM', 'natural', 'ER'),
    ('GBM', 'AP', 'acc.'), ('GBM', 'AP', 'ER'), ('GBM', 'AP', '\\defscore'),
])
table.columns = pd.MultiIndex.from_tuples(table.columns, names=['model', 'method', ''])
table

./results/fullmnist-pca100-xgb-xgb-models-prune-fullmnist-pca100-20-rf-linf.0200.model-blackbox-rs0-linf.json doesn't exist
./results/fullmnist-pca100-xgb-xgb-models-fullmnist-pca100.0200.model-blackbox-rs0-linf.json doesn't exist
./results/fullfashion-pca100-xgb-xgb-models-prune-fullfashion-pca100-20-rf-linf.0200.model-blackbox-rs0-linf.json doesn't exist
./results/fullfashion-pca100-xgb-xgb-models-fullfashion-pca100.0200.model-blackbox-rs0-linf.json doesn't exist


model            RF                                 $1$-NN                \
method      natural            AP                  natural            AP   
               acc.     ER   acc.     ER \defscore    acc.     ER   acc.   
fullfashion   0.825  0.182  0.790  0.171     0.936   0.870  0.196  0.815   
fullmnist     0.950  0.141  0.935  0.152     1.080   0.935  0.221  0.940   

model                         $3$-NN                                    GBM  \
method                       natural            AP                  natural   
                ER \defscore    acc.     ER   acc.     ER \defscore    acc.   
fullfashion  0.225     1.148   0.865  0.217  0.835  0.242     1.119   0.885   
fullmnist    0.222     1.007   0.950  0.233  0.940  0.269     1.155   0.980   

model                                       
method                 AP                   
                ER   acc.     ER \defscore  
fullfashion  0.161  0.860  0.164     1.013  
fullmnist    0.185  0.965  0.182     0.981

In [7]:
_, exp_name, grid_params, _ = params_l2.fullds()()
df = params_to_dataframe(grid_params, columns=[("tst_score"), ("avg_pert"), "aug_len"])
table = {}
model_names = ['random_forest_300_d10', 'approxAP_rf_300_500_d10',
               'knn1', 'approxAP_nn_k1_500',
               'knn3', 'approxAP_nn_k3_500']
shown_names = ['RF', 'AP RF', '$1$-NN', 'AP $1$-NN', '$3$-NN', 'AP $3$-NN']
for ds_name_t, d in df.groupby("dataset"):
    ds_name = ds_name_t.split("_")[0]
 
    table[ds_name] = []
    for i, model_name in enumerate(model_names):
        model_df = d[d['model'] == model_name]
        if len(model_df['tst_score'].values) == 0:
            table[ds_name].append("%.3f" % -1)
            table[ds_name].append("%.3f" % -1)
            if i % 2:
                table[ds_name].append("%.3f" % -1)
        else:
            table[ds_name].append("%.3f" % model_df['tst_score'].values[0])
            table[ds_name].append("%.3f" % model_df['avg_pert'].values[0])
            if i % 2:
                try:
                    table[ds_name].append(
                        "%.3f" % (model_df['avg_pert'].values[0] / d[d['model'] == model_names[i-1]]['avg_pert'].values[0]))
                except:
                    table[ds_name].append("%.3f" % -1)


table = pd.DataFrame.from_dict(table, orient='index', columns=[
    ('RF', 'natural', 'acc.'), ('RF', 'natural', 'ER'),
    ('RF', 'AP', 'acc.'), ('RF', 'AP', 'ER'), ('RF', 'AP', '\\defscore'),
    ('$1$-NN', 'natural', 'acc.'), ('$1$-NN', 'natural', 'ER'),
    ('$1$-NN', 'AP', 'acc.'), ('$1$-NN', 'AP', 'ER'), ('$1$-NN', 'AP', '\\defscore'),
    ('$3$-NN', 'natural', 'acc.'), ('$3$-NN', 'natural', 'ER'),
    ('$3$-NN', 'AP', 'acc.'), ('$3$-NN', 'AP', 'ER'), ('$3$-NN', 'AP', '\\defscore'),
])
table.columns = pd.MultiIndex.from_tuples(table.columns, names=['model', 'method', ''])
table

./results/fullmnist-random-forest-500-d10-blackbox-rs0-l2.json doesn't exist
./results/fullmnist-approxAP-nn-k1-600-blackbox-rs0-l2.json doesn't exist
./results/fullmnist-approxAP-nn-k3-600-blackbox-rs0-l2.json doesn't exist
./results/fullfashion-knn3-blackbox-rs0-l2.json doesn't exist


model            RF                                  $1$-NN                \
method      natural             AP                  natural            AP   
               acc.      ER   acc.     ER \defscore    acc.     ER   acc.   
fullfashion   0.860   0.679  0.840  0.907     1.336   0.880  2.273  0.855   
fullmnist    -1.000  -1.000  0.975  0.823    -1.000   0.965  2.597  0.970   

model                         $3$-NN                                  
method                       natural             AP                   
                ER \defscore    acc.      ER   acc.     ER \defscore  
fullfashion  2.752     1.211  -1.000  -1.000  0.830  2.885    -1.000  
fullmnist    2.612     1.006   0.970   3.066  0.975  3.050     0.995

In [15]:
%%bash
bash ./sync_report.sh

In [1]:
assert 1==0

AssertionError: 

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.spatial import Voronoi, voronoi_plot_2d


def draw_boundary(clf, X, y, file_name=None):
    h = .01  # step size in the mesh
    
    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
    #cmap_light = ListedColormap(['#ffffff', '#43a2ca'])
    #cmap_bold = ListedColormap(['#e0f3db', '#a8ddb5'])

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 0.25, X[:, 0].max() + 0.25
    y_min, y_max = X[:, 1].min() - 0.25, X[:, 1].max() + 0.25
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    fig = plt.figure(figsize=(8, 8))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.axis('off')
    plt.title("")
    
    if file_name is not None:
        plt.savefig(file_name, transparent=True)
    plt.show()
    return fig

ModuleNotFoundError: No module named 'pvn'

In [9]:
X, y = datasets.make_moons(n_samples=1000, noise=0.20, random_state=0)
clf = neighbors.KNeighborsClassifier(1).fit(X, y)
_ = draw_boundary(clf, X, y, "figs/moon_1nn.png")

NameError: name 'draw_boundary' is not defined

In [21]:
a = json.load(open("./results/fullmnist-approxAP-faisslshknn-3-500-500-blackbox-rs0-l2.json", "r"))
b = json.load(open("./results/fullmnist-faisslshknn-3-500-blackbox-rs0-l2.json", "r"))
print(a['avg_pert']['avg'], b['avg_pert']['avg'], a['avg_pert']['avg'] / b['avg_pert']['avg'])

2.923543828315131 2.92527918559083 0.9994067720837564
